# Polarizability using automatic differentiation

Simple example for computing properties using (forward-mode)
automatic differentiation.
For a more classical approach and more details about computing polarizabilities,
see Polarizability by linear response.

In [1]:
using DFTK
using LinearAlgebra
using ForwardDiff

# Construct PlaneWaveBasis given a particular electric field strength
# Again we take the example of a Helium atom.
function make_basis(ε::T; a=10., Ecut=30) where {T}
    lattice=T(a) * I(3)  # lattice is a cube of $a$ Bohrs
    # Helium at the center of the box
    atoms     = [ElementPsp(:He, psp=load_psp("hgh/lda/He-q2"))]
    positions = [[1/2, 1/2, 1/2]]

    model = model_DFT(lattice, atoms, positions, [:lda_x, :lda_c_vwn];
                      extra_terms=[ExternalFromReal(r -> -ε * (r[1] - a/2))],
                      symmetries=false)
    PlaneWaveBasis(model; Ecut, kgrid=[1, 1, 1])  # No k-point sampling on isolated system
end

# dipole moment of a given density (assuming the current geometry)
function dipole(basis, ρ)
    @assert isdiag(basis.model.lattice)
    a  = basis.model.lattice[1, 1]
    rr = [a * (r[1] - 1/2) for r in r_vectors(basis)]
    sum(rr .* ρ) * basis.dvol
end

# Function to compute the dipole for a given field strength
function compute_dipole(ε; tol=1e-8, kwargs...)
    scfres = self_consistent_field(make_basis(ε; kwargs...); tol)
    dipole(scfres.basis, scfres.ρ)
end;

With this in place we can compute the polarizability from finite differences
(just like in the previous example):

In [2]:
polarizability_fd = let
    ε = 0.01
    (compute_dipole(ε) - compute_dipole(0.0)) / ε
end

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770866520021                   -0.53    8.0         
  2   -2.772142170334       -2.89       -1.31    1.0    154ms
  3   -2.772170188308       -4.55       -2.61    1.0    165ms
  4   -2.772170718029       -6.28       -3.79    2.0    216ms
  5   -2.772170722453       -8.35       -4.18    2.0    180ms
  6   -2.772170723007       -9.26       -5.61    1.0    199ms
  7   -2.772170723015      -11.10       -5.97    2.0    186ms
  8   -2.772170723015      -12.90       -6.61    1.0    171ms
  9   -2.772170723015      -13.45       -7.19    2.0    214ms
 10   -2.772170723015   +  -14.12       -7.75    1.0    181ms
 11   -2.772170723015      -14.03       -8.73    2.0    213ms
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770744352318                   -0.52    8.0         
  2   -2.7

1.7735580351577616

We do the same thing using automatic differentiation. Under the hood this uses
custom rules to implicitly differentiate through the self-consistent
field fixed-point problem.

In [3]:
polarizability = ForwardDiff.derivative(compute_dipole, 0.0)
println()
println("Polarizability via ForwardDiff:       $polarizability")
println("Polarizability via finite difference: $polarizability_fd")

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770725173360                   -0.53    8.0         
  2   -2.772043844119       -2.88       -1.30    1.0    153ms
  3   -2.772081863915       -4.42       -2.65    1.0    214ms
  4   -2.772083414576       -5.81       -3.98    2.0    177ms
  5   -2.772083417747       -8.50       -4.65    2.0    243ms
  6   -2.772083417807      -10.22       -5.67    1.0    162ms
  7   -2.772083417811      -11.51       -6.26    2.0    184ms
  8   -2.772083417811      -13.34       -7.27    2.0    220ms
  9   -2.772083417811      -15.35       -8.02    2.0    201ms

Polarizability via ForwardDiff:       1.772534975268487
Polarizability via finite difference: 1.7735580351577616
